# Notebook walkthrough of using the Basic portion of Kiwiglider to process realtime Slocum glider data and create summary pages for delayed mode Slocum glider data

# Set up kiwiglider

## C-compiler

You will need a C-compiler to use kiwiglider, since it relies on a package called dbdreader. See dbdreader installation suggestions for [Linux](https://github.com/smerckel/dbdreader?tab=readme-ov-file#installation-linux) and [Windows](https://github.com/smerckel/dbdreader?tab=readme-ov-file#installation-on-windows) for information about C-compilers (note that you do not need to install dbdreader itself at this stage)

## Environment

We recommend creating a separate environment for your work with kiwiglider.

1. `conda create -n kiwiglider`
1. `conda activate kiwiglider`

## Install

As of late May 2025, one dependency fails under pip but works with conda. As such, also do `conda install compliance-checker cc-plugin-glider`

### Method 1

From your new virtual environment, you can now install kiwiglider. The easiest method as of late May 2025 is with

`pip install --index-url https://test.pypi.org/simple/ --extra-index-url https://test.pypi.org/simple/ kiwiglider`

### Method 2

Another method of installation is through git and pip. This will allow you to edit kiwiglider in a local repository. After you have [Git](https://git-scm.com/downloads) installed on your machine: 

1. Clone kiwiglider to your local machine: `git clone https://github.com/adeverne/kiwiglider`
1. Change to parent directory of kiwiglider
1. Install kiwiglider with `pip install -e ./kiwiglider`

## Notebook

To run this notebook, you will also need to `conda install ipykernel`

# Load kiwiglider functions and classes

In [ ]:
#general imports
from os.path import join as join_path
from os.path import exists
from os import sep,makedirs
from shutil import copy
import logging
#kiwiglider imports
from kiwiglider.basic import DeploymentYAML,DeploymentNetCDF

# Initialize

Start the logging package, which will keep us updated during processing

In [ ]:
logging.basicConfig(level='INFO')

Specify the directory where all the organization's deployments and metadata live. In NIWA's case, and before running this notebook, the main directory is organized with


- deployments_NIWA.xlsx
- GLD0039
    - Raw
        - Cache
        - *.sbd
        - *.tbd
        - *.dbd
        - *.ebd
- GLD0040
    - Raw
        - Cache
        - *.sbd
        - *.tbd
        - *.dbd
        - *.ebd


and so on for all deployments

In [ ]:
directory = join_path('C:',sep,'Users','elmerc','OneDrive - NIWA','Documents','SCRATCH')

Specify the deployment name and the summary to add in IOOS GliderDAC NetCDF files

In [ ]:
deployment_ID = 40
deployment = 'GLD{:04d}'.format(deployment_ID)
summary = 'Slocum glider dataset gathered as part of the OCOF project at NIWA. This dataset contains physical oceanographic measurements of temperature, conductivity, salinity, density and estimates of depth-average currents.'

# Create YAML file for pyglider ingest

In [ ]:
yml = DeploymentYAML(ID=deployment_ID)
yml.construct_yaml(excelsheet=join_path(directory,'deployments_NIWA.xlsx'),metadata={'summary':summary})
yml.write_yaml(outname=join_path(directory,deployment,'deployment_metadata.yml'))

We'll also copy this file into Realtime and Delayed directories. Kiwiglider is set up to ingest YAMLs from the same directory as the processed output files (allowing the user to specify different parameters for realtime and delayed modes), so if you wish to use the same YAML for realtime and delayed data it will need to be copied or you will need to specify it within the initialization of the DeploymentNetCDF class (below).

In [ ]:
if not exists(join_path(directory,deployment,'Realtime')):
    makedirs(join_path(directory,deployment,'Realtime'))
copy(join_path(directory,deployment,'deployment_metadata.yml'),join_path(directory,deployment,'Realtime'))
if not exists(join_path(directory,deployment,'Delayed')):
    makedirs(join_path(directory,deployment,'Delayed'))
copy(join_path(directory,deployment,'deployment_metadata.yml'),join_path(directory,deployment,'Delayed'))

# Create realtime files (eventually will send to IOOS GliderDAC)

This is the recommended use of kiwiglider basic, since kiwiglider steps contains a more thourough post-processing regiment for delayed mode data.

In this notebook, all realtime files are available. However, during normal use, this section will need to be run every time there is new data available.

## Initialize

In [ ]:
realtime = DeploymentNetCDF(main_directory=join_path(directory,deployment),style='Realtime')

## Create L0 (converted timeseries only)

While you can create individual profiles here (it is, in fact, the default) we will skip it for faster processing and smaller directory footprint.

In [ ]:
realtime.make_L0(l0profile_directory=None)

## Create L1 (with QARTOD tests)

In [ ]:
realtime.make_L1()

## Check IOOS GliderDAC compliance

In [ ]:
realtime.check_compliance()

# Create delayed mode summary

While not recommended for data disemination purposes, using kiwiglider basic represents a "quick and dirty" processing path. The main purpose for using kiwiglider basic is to produce a mission summary page, as follows.

## Initialize

In [ ]:
delayed = DeploymentNetCDF(main_directory=join_path(directory,deployment),style='Delayed')

## Create L0 (timeseries only)

While you can create individual profiles here (it is, in fact, the default) we will skip it for faster processing and smaller directory footprint.

In [ ]:
delayed.make_L0(l0profile_directory=None)

## Create summary page

In [ ]:
delayed.create_summary(author='Cassandra Elmer')